# IMPORT LIBRARY

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import requests
import time
import re

## DEFINE TABLE

In [2]:
title = []
content = []
amount = []
date = []
clock = []
alllink = []

# OPEN WEB BROWSER AND SCROLL DOWN

In [12]:
options = Options()
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
driver = webdriver.Chrome(executable_path=r'C:\Users\tanna\OneDrive\เดสก์ท็อป\DSI2022\chromedriver.exe', options=options)
driver.get('https://pantip.com/search?q=%E0%B8%AD%E0%B8%B2%E0%B8%AB%E0%B8%B2%E0%B8%A3%E0%B8%AD%E0%B8%AD%E0%B9%81%E0%B8%81%E0%B8%99%E0%B8%B4%E0%B8%84&rooms=food,family,greenzone')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

## COLLECT LINK

In [13]:
data = driver.page_source
soup = bs(data)
soup2 = bs(str(soup.findAll("div",{'class':'m-b-20'})))

In [14]:
for i in soup2.findAll('a'):
    i = str(i.get('href'))
    b = re.search("topic",i)
    if b != None:
        i = re.sub("/comment\d.*","",i)
        alllink.append(i)

In [ ]:
alllist = list(dict.fromkeys(alllist))

## START SCRAPE FROM EACH LINK

In [15]:
#Open browser ด้วย Selenium
for i in alllink:
    options = Options()
    options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
    driver = webdriver.Chrome(executable_path=r'C:\Users\tanna\OneDrive\เดสก์ท็อป\DSI2022\chromedriver.exe', options=options)
    driver.get(i)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    
    data = driver.page_source
    soup = bs(data)
    
    soupcontent = soup.findAll('div',{'class':'display-post-story'})
    souptitle = soup.findAll('h2',{'class':'display-post-title'})
    soupallcomment = soup.findAll('span',{'class':'title'})
    souptime = soup.findAll('span',{'class':'display-post-timestamp'})
    
    for i in souptitle:
        b = i.get_text()
        title.append(b)
        
    text = []
    for i in soupcontent:
        b = i.get_text()
        b = str(b)
        b = b.replace("\t","")
        b = b.replace("\n","")
        b = b.replace("\xa0","")
        text.append(b)
    content.append(text)
    
    comamount = soupallcomment[0].get_text()
    comamount = re.findall("\d",comamount)
    comamount = "".join(comamount)
    amount.append(comamount)
    
    a = souptime[0]
    a = str(a).replace("data-utime","aaa")
    a = bs(str(a))
    find = a.findAll("abbr")
    for i in find:
        i = i.get('aaa').split()
        date.append(i[0])
        clock.append(i[1])
        
    driver.quit()

# START SCRAPE

In [ ]:
options = Options()
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
driver = webdriver.Chrome(executable_path=r'C:\Users\tanna\OneDrive\เดสก์ท็อป\DSI2022\chromedriver.exe', options=options)
driver.get('https://pantip.com/topic/32776340')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
data = driver.page_source

In [ ]:
soup = bs(data)

In [ ]:
soupcontent = soup.findAll('div',{'class':'display-post-story'})
souptitle = soup.findAll('h2',{'class':'display-post-title'})
soupallcomment = soup.findAll('span',{'class':'title'})
souptime = soup.findAll('span',{'class':'display-post-timestamp'})


### LINK

In [11]:
alllink.append('https://pantip.com/topic/32776340')

### TITLE

In [ ]:
for i in souptitle:
    b = i.get_text()
    title.append(b)

### CONTENT

In [ ]:
text = []
for i in soupcontent:
    b = i.get_text()
    b = str(b)
    b = b.replace("\t","")
    b = b.replace("\n","")
    b = b.replace("\xa0","")
    text.append(b)
content.append(text)

### COMMENT AMOUNT

In [ ]:
comamount = soupallcomment[0].get_text()
comamount = re.findall("\d",comamount)
comamount = "".join(comamount)
amount.append(comamount)

### TIME

In [ ]:
a = souptime[0]
a = str(a).replace("data-utime","aaa")
a = bs(str(a))
find = a.findAll("abbr")
for i in find:
    i = i.get('aaa').split()
    date.append(i[0])
    clock.append(i[1])

# CONVERT TO DATAFRAME AND PUBLISH

In [16]:
final = pd.DataFrame({"Topic": title,
                     "ComAmount": amount,
                     "Comment": content,
                     "Date": date,
                     "Time": clock,
                     "URL": alllink})

final.ComAmount = final.ComAmount.replace("",0)
final = final.drop_duplicates(subset=['URL'])

In [18]:
final.to_csv('pantipscrap.csv',index = False)

In [19]:
final

,Topic,ComAmount,Comment,Date,Time,URL
0,ทำไม? ต้องออร์แกนิค,0,[หลายคนสงสัยนะครับ ว่าทำไมการเลือกกินเลือกใช้ผ...,08/26/2016,17:45:05,https://pantip.com/topic/35530303
1,ผักออร์แกนิค ทำไมแพงจัง มีวิธีการทำให้มันถูกมั...,4,[คือตอนนี้เริ่มจะสนใจผักออร์แกนิค หมักปุ๋ย แต่...,02/17/2016,12:23:19,https://pantip.com/topic/34804839
2,อยากรู้จัง ว่าสินค้าที่มีการบอกว่าตยเองเป็น อ...,11,[ตอนนี้เห็นหลายสินค้าออกมาบอกว่าตนเองเป็นสินค้...,02/24/2014,10:50:50,https://pantip.com/topic/31698857
3,ปกติสั่งอาหาร organic ที่ไหนกันบ้างคะ?,0,[พอดีเป็นโรคภูมิแพ้และสนใจอาหารออแกนิคมากค่ะอย...,02/06/2017,15:53:18,https://pantip.com/topic/36085384
4,ออร์แกนิค....อาหารเพื่อสุขภาพที่ดี,2,[ออร์แกนิค....อาหารเพื่อสุขภาพที่ดีเพราะสุขภาพ...,07/15/2014,10:12:26,https://pantip.com/topic/32325362
5,แหล่งพืชออร์แกนิคในเขต กทม. ????,0,[พอมีใครทราบพื้นที่เพาะปลูก หรือว่าแหล่งขายพืช...,09/03/2014,15:17:41,https://pantip.com/topic/32533934
6,ผักออร์แกนิคกับผักไฮโดรโปนิกส์ ต่างกันอย่างไร ...,32,[ก่อนอื่นเราอยากบอกว่าเราศึกษาเปรียบเทียบผักออ...,08/19/2013,20:37:33,https://pantip.com/topic/30864043
7,[เกษตรอินทรีย์] ออร์แกนิค คืออะไร ทำไมต้องออร์...,0,[ออร์แกนิค คืออะไร ทำไมต้องออร์แกนิคในปัจจุบัน...,04/17/2016,12:00:55,https://pantip.com/topic/35045051
8,ผักออแกนิคที่ขายตามห้างนี่คืออะไรคะ?,0,[ตอนแรกเราคิดว่าผักออแกนิคคือผักปลอดสารและโตธร...,10/28/2014,13:17:25,https://pantip.com/topic/32776340
9,ผักออแกนิคในร้านโอ้กะจู๋มีไว้สำหรับจัดจานให้สว...,0,[เคยไปทานอาหารกับรุ่นพี่ที่ร้านโอ้กะจู๋ซึ่งไปก...,04/11/2022,17:23:31,https://pantip.com/topic/41369458
